In [2]:
import numpy as np
from scipy.optimize import minimize

def continuous_reference_set_method(objective_functions, directions, a, b, bounds, x0=None):
    """
    Implementacja metody zbiorów odniesienia dla ciągłego problemu optymalizacji wielokryterialnej.

    Parametry:
    - objective_functions: lista funkcji celu
    - directions: numpy array z wartościami 1 (maksymalizacja) lub -1 (minimalizacja) dla każdego kryterium
    - a: numpy array z punktami aspiracji dla każdego kryterium
    - b: numpy array z punktami status quo dla każdego kryterium
    - bounds: lista krotek określających ograniczenia dla zmiennych decyzyjnych
    - x0: opcjonalnie, punkt startowy dla optymalizacji

    Zwraca:
    - result: wynik optymalizacji z biblioteki scipy.optimize
    - optimal_f_values: wartości funkcji celu w optymalnym punkcie
    - optimal_score: wartość funkcji skoringowej w optymalnym punkcie
    """

    # Liczba kryteriów
    num_criteria = len(objective_functions)
    
    # Funkcja agregująca wartości funkcji celu
    def F(x):
        return np.array([f(x) for f in objective_functions])
    
    # Funkcja normalizująca z uwzględnieniem kierunku optymalizacji
    def normalize(f_values, a, b, directions):
        norm_values = np.zeros_like(f_values)
        for i in range(len(f_values)):
            if directions[i] == -1:  # Minimalizacja
                norm_values[i] = (f_values[i] - a[i]) / (b[i] - a[i])
            else:  # Maksymalizacja
                norm_values[i] = (b[i] - f_values[i]) / (b[i] - a[i])
        return norm_values

    # Funkcja skoringowa oparta na metodzie TOPSIS
    def scoring_function(x):
        f_values = F(x)
        # Normalizacja z uwzględnieniem kierunku optymalizacji
        f_norm = normalize(f_values, a, b, directions)
        # Obliczanie odległości od punktu idealnego (aspiracji) i anty-idealnego (status quo)
        distance_to_a = np.linalg.norm(f_norm - np.zeros(len(f_norm)))
        distance_to_b = np.linalg.norm(f_norm - np.ones(len(f_norm)))
        # Wskaźnik bliskości do ideału
        C = distance_to_b / (distance_to_a + distance_to_b)
        return C

    # Funkcja do minimalizacji (negacja wskaźnika C)
    def objective(x):
        return -scoring_function(x)
    
    # Punkt startowy
    if x0 is None:
        x0 = np.zeros(len(bounds))
    
    # Optymalizacja
    result = minimize(
        objective,
        x0=x0,
        bounds=bounds,
        method='SLSQP'
    )
    
    optimal_x = result.x
    optimal_f_values = F(optimal_x)
    optimal_score = scoring_function(optimal_x)
    
    return result, optimal_f_values, optimal_score

In [3]:
import numpy as np

def discrete_reference_set_method(alternatives, directions, a, b):
    """
    Implementacja metody zbiorów odniesienia dla dyskretnego problemu optymalizacji wielokryterialnej.
    
    Parametry:
    - alternatives: numpy array z listą alternatyw (rozwiązań), gdzie każdy wiersz to alternatywa, a kolumny to kryteria.
    - directions: numpy array z wartościami 1 (maksymalizacja) lub -1 (minimalizacja) dla każdego kryterium.
    - a: numpy array z punktami aspiracji (najlepsze wartości) dla każdego kryterium.
    - b: numpy array z punktami status quo (najgorsze wartości) dla każdego kryterium.
    
    Zwraca:
    - ranking: numpy array z indeksami alternatyw posortowanych od najbardziej preferowanych do najmniej.
    - scores: numpy array z wartościami funkcji skoringowej dla każdej alternatywy.
    """
    
    # Funkcja normalizująca z uwzględnieniem kierunku optymalizacji
    def normalize(alternatives, a, b, directions):
        norm_alt = np.zeros_like(alternatives, dtype=float)
        for i in range(len(directions)):
            if directions[i] == -1:  # Minimalizacja
                norm_alt[:, i] = (alternatives[:, i] - a[i]) / (b[i] - a[i])
            else:  # Maksymalizacja
                norm_alt[:, i] = (b[i] - alternatives[:, i]) / (b[i] - a[i])
        return norm_alt
    
    # Funkcja skoringowa
    def scoring_function(alternatives, a, b, directions):
        norm_alt = normalize(alternatives, a, b, directions)
        distance_to_a = np.linalg.norm(norm_alt - np.zeros((len(alternatives), len(a))), axis=1)
        distance_to_b = np.linalg.norm(norm_alt - np.ones((len(alternatives), len(a))), axis=1)
        C = distance_to_b / (distance_to_a + distance_to_b)
        return C
    
    # Obliczanie wartości funkcji skoringowej
    scores = scoring_function(alternatives, a, b, directions)
    
    # Tworzenie rankingu
    ranking = np.argsort(-scores)  # Negacja, aby sortować malejąco
    
    return ranking, scores
